# Load Drive and Depedency

In [2]:
from google.colab import drive
drive.mount('/content/drive')
# CHANGE BASED ROOT DIR
%cd '/content/drive/MyDrive/Image Retrieval'

ModuleNotFoundError: No module named 'google.colab'

## Depedency Library

In [1]:
%pip install numpy opencv-python scikit-learn pandas matplotlib tensorflow

     --------------------------------------- 12.9/12.9 MB 12.6 MB/s eta 0:00:00
     --------------------------------------- 39.5/39.5 MB 13.6 MB/s eta 0:00:00
     --------------------------------------- 11.1/11.1 MB 17.2 MB/s eta 0:00:00
     --------------------------------------- 11.6/11.6 MB 18.2 MB/s eta 0:00:00
     ---------------------------------------- 8.1/8.1 MB 17.8 MB/s eta 0:00:00
     -------------------------------------- 375.9/375.9 MB 7.4 MB/s eta 0:00:00
     --------------------------------------- 41.2/41.2 MB 21.8 MB/s eta 0:00:00
     ------------------------------------- 301.8/301.8 kB 18.2 MB/s eta 0:00:00
     ------------------------------------- 509.2/509.2 kB 33.3 MB/s eta 0:00:00
     ------------------------------------- 347.8/347.8 kB 21.1 MB/s eta 0:00:00
     ---------------------------------------- 219.8/219.8 kB ? eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 28.1 MB/s eta 0:00:00
     ---------------------------------------- 7


[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import json
import os
import cv2
import copy
import sklearn.metrics as metric
import pandas as pd

from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint 

import matplotlib.pyplot as plt
import matplotlib

# Training Stage

## Core Model AutoEncoder

In [3]:
class ConvAutoEncoder:
    """
    Core Convolutional AutoEncoder
    This class included with build method you can adjust of width height and depth of image
    default filters 128, and latent dimension 48
    """
    @staticmethod
    def build(width, height, depth, filters=(128,), latent_dim=48):
        input_shape = (height, width, depth)
        channel_dim = -1
        inputs = layers.Input(shape=input_shape)
        x = inputs
        # Encoder layer
        for f in filters:
            x = layers.Conv2D(f, (3, 3), strides=2, padding='same')(x)
            x = layers.LeakyReLU(negative_slope=0.2)(x)
            x = layers.BatchNormalization(axis=channel_dim, name='enc_filter_' + str(f))(x)
        volume_size = K.int_shape(x)
        x = layers.Flatten()(x)
        # Latent layer
        latent = layers.Dense(latent_dim, name="encoded")(x)
        # Decoder layer
        x = layers.Dense(np.prod(volume_size[1:]))(latent)
        x = layers.Reshape((volume_size[1], volume_size[2], volume_size[3]))(x)
        # Reverse on decoder
        for f in filters[::-1]:
            x = layers.Conv2DTranspose(f, (3, 3), strides=2, padding='same')(x)
            x = layers.LeakyReLU(negative_slope=0.2)(x)
            x = layers.BatchNormalization(axis=channel_dim, name='dec_filter_' + str(f))(x)
        x = layers.Conv2DTranspose(depth, (3, 3), padding="same")(x)
        outputs = layers.Activation("sigmoid", name="decoded")(x)
        auto_encoder = Model(inputs, outputs, name="auto_encoder")
        return auto_encoder

## Definition of hyperparameter


In [4]:
EPOCHS = 20
INIT_LR = 1e-3
BS = 16
IMAGE_SIZE = (256, 256)
checkpoint_path = "training_1_40_binary_cp/cp.ckpt"
feature_path = "training_1_40_binary_feature.json"
model_path = "training_1_40_binary_model.h5"
history_path = "training_1_40_binary_history.json"
magnification = "40X"
base_dataset = "binary_scenario"
class_dir = ['benign', 'malignant']

## Load Image


```
# Load all available file first then process to load images into variable
```



In [14]:
print("[INFO] indexing file images BreaKHis dataset...")
type_dataset = ['val', 'train']
dataset_train = []
dataset_val = []
for type_set in type_dataset:
    for class_item in class_dir:
        cur_dir = os.path.join(base_dataset, type_set, magnification ,class_item)
        for file in os.listdir(cur_dir):
            if type_set == 'train':
                dataset_train.append(os.path.join(cur_dir, file))
            else:
                dataset_val.append(os.path.join(cur_dir, file))

print("[INFO] load images BreaKHis dataset...")
#  load images
train_images = []
val_images = []
for type_set in type_dataset:
    cur_dataset = dataset_train if type_set == 'train' else dataset_val
    for image_path in cur_dataset:
        if ".png" in image_path:
            image = cv2.imread(image_path)
            image = cv2.resize(image, IMAGE_SIZE)
            if type_set == 'train':
                train_images.append(image)
            else:
                val_images.append(image)
                
print(dataset_train)
print(dataset_val)

[INFO] indexing file images BreaKHis dataset...
[INFO] load images BreaKHis dataset...
['binary_scenario\\train\\40X\\benign\\SOB_B_A-14-22549AB-40-002.png', 'binary_scenario\\train\\40X\\benign\\SOB_B_A-14-22549AB-40-003.png', 'binary_scenario\\train\\40X\\benign\\SOB_B_A-14-22549AB-40-004.png', 'binary_scenario\\train\\40X\\benign\\SOB_B_A-14-22549AB-40-005.png', 'binary_scenario\\train\\40X\\benign\\SOB_B_A-14-22549AB-40-006.png', 'binary_scenario\\train\\40X\\benign\\SOB_B_A-14-22549AB-40-008.png', 'binary_scenario\\train\\40X\\benign\\SOB_B_A-14-22549AB-40-009.png', 'binary_scenario\\train\\40X\\benign\\SOB_B_A-14-22549AB-40-011.png', 'binary_scenario\\train\\40X\\benign\\SOB_B_A-14-22549AB-40-012.png', 'binary_scenario\\train\\40X\\benign\\SOB_B_A-14-22549AB-40-013.png', 'binary_scenario\\train\\40X\\benign\\SOB_B_A-14-22549AB-40-014.png', 'binary_scenario\\train\\40X\\benign\\SOB_B_A-14-22549AB-40-015.png', 'binary_scenario\\train\\40X\\benign\\SOB_B_A-14-22549AB-40-016.png', 'b

## Normalization

In [6]:
# normalization
print("[INFO] normalization...")
train_x = np.array(train_images).astype("float32") / 255.0
val_x = np.array(val_images).astype("float32") / 255.0

[INFO] normalization...


## Buiding Model based ConvAutoEncoder Class

In [8]:
print("[INFO] building auto encoder...")
auto_encoder = ConvAutoEncoder.build(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)
opt = Adam(learning_rate=INIT_LR)
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path + ".keras",
    monitor="val_loss",
    verbose = 1,
    mode='min',
    save_best_only=True)
auto_encoder.compile(loss="mse", optimizer=opt)
auto_encoder.summary()

[INFO] building auto encoder...


Model: "auto_encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 128)  │         3,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enc_filter_128                  │ (None, 128, 128, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2097152)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoded (Dense)                 │ (None, 48)             │   100,663,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2097152)        │   102,760,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_1 (Reshape)             │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_2              │ (None, 256, 256, 128)  │       147,584 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 256, 256, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dec_filter_128                  │ (None, 256, 256, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_3              │ (None, 256, 256, 3)    │         3,459 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoded (Activation)            │ (None, 256, 256, 3)    │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 203,579,443 (776.59 MB)

 Trainable params: 203,578,931 (776.59 MB)

 Non-trainable params: 512 (2.00 KB)

## Training The Model

In [9]:
# train the convolutional auto encoder
print("[INFO] training auto encoder...")
H = auto_encoder.fit(
    train_x, train_x,
    shuffle=True,
    validation_data=(val_x, val_x),
    epochs=EPOCHS,
    callbacks = [checkpoint_callback], batch_size=BS)

[INFO] training auto encoder...
Epoch 1/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 0.0715
Epoch 1: val_loss improved from inf to 0.01686, saving model to training_1_40_binary_cp/cp.ckpt.keras
102/102 ━━━━━━━━━━━━━━━━━━━━ 460s 4s/step - loss: 0.0712 - val_loss: 0.0169
Epoch 2/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - loss: 0.0115
Epoch 2: val_loss did not improve from 0.01686
102/102 ━━━━━━━━━━━━━━━━━━━━ 467s 5s/step - loss: 0.0115 - val_loss: 0.0179
Epoch 3/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 0.0108
Epoch 3: val_loss did not improve from 0.01686
102/102 ━━━━━━━━━━━━━━━━━━━━ 461s 5s/step - loss: 0.0108 - val_loss: 0.0208
Epoch 4/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 0.0106
Epoch 4: val_loss did not improve from 0.01686
102/102 ━━━━━━━━━━━━━━━━━━━━ 460s 5s/step - loss: 0.0106 - val_loss: 0.0199
Epoch 5/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 0.0104
Epoch 5: val_loss did not improve from 0.01686
102/102 ━━━━━━━━━━━━━━━━━━━━ 460s 5s/step - 

## Saved Result Graphic (.json) and Model Trained (h5)

In [10]:
with open(history_path, 'w') as f:
    json.dump(H.history, f)
auto_encoder.save(model_path)

# Extract Feature Stage

## Load already trained model and extracting feature


In [12]:
auto_encoder = ConvAutoEncoder.build(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)
# load our auto_encoder from disk
print("[INFO] loading auto encoder model...")
auto_encoder.load_weights(checkpoint_path + ".keras")

# create the encoder model which consists of *just* the encoder
# portion of the auto encoder
encoder = Model(inputs=auto_encoder.input,
	outputs=auto_encoder.get_layer("encoded").output)

# quantify the contents of our input images using the encoder
print("[INFO] encoding images...")
features = encoder.predict(train_x)

[INFO] loading auto encoder model...
[INFO] encoding images...
51/51 ━━━━━━━━━━━━━━━━━━━━ 7s 141ms/step


## Create mapping feature included location images, label, feature and indexing number

In [16]:
indexes = list(range(0, train_x.shape[0]))
features_array = [[float(x) for x in y] for y in features]
labels = [path.split("\\")[3] for path in dataset_train]
data = {"indexes": indexes, "features": features_array, "locations": dataset_train, "labels":labels}

## Saved model feature (already mapped) as json

In [17]:
with open(feature_path, 'w') as f:
    json.dump(data, f)

# Retrieval Test Stage

## Euclidean function

In [18]:
def euclidean(a, b):
	# compute and return the euclidean distance between two vectors
	return np.linalg.norm(a - b)

## Perform search function
```
Default max result from search is 5
```




In [19]:
def perform_search(query_features, indexed_train, max_results=5):
	retrieved = []
	for idx in range(0, len(indexed_train["features"])):
		distance = euclidean(query_features, indexed_train["features"][idx])
		retrieved.append((distance, idx))
	retrieved = sorted(retrieved)[:max_results]
	return retrieved

## Load test images

In [20]:
print("[INFO] indexing file images BreaKHis dataset...")
# indexing file images
dataset = []
for class_item in class_dir:
    cur_dir = os.path.join(base_dataset, 'test', magnification, class_item)
    for file in os.listdir(cur_dir):
        dataset.append(os.path.join(cur_dir, file))
print("test len to retrieving:", len(dataset))
print("[INFO] load test images BreaKHis dataset...")
#  load images
images = []
for image_path in dataset:
    if ".png" in image_path:
        image = cv2.imread(image_path)
        image = cv2.resize(image, IMAGE_SIZE)
        images.append(image)

[INFO] indexing file images BreaKHis dataset...
test len to retrieving: 199
[INFO] load test images BreaKHis dataset...


## Normalization

In [21]:
# normalization
print("[INFO] normalization...")
test_x = np.array(images).astype("float32") / 255.0

[INFO] normalization...


## Load the model and feature already extracted

In [23]:
auto_encoder = ConvAutoEncoder.build(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)
# load our auto_encoder from disk
print("[INFO] loading auto encoder model...")
auto_encoder.load_weights(checkpoint_path + ".keras")
with open(feature_path) as f:
  training_indexed = json.load(f)

# create the encoder model which consists of *just* the encoder
# portion of the auto encoder
encoder = Model(inputs=auto_encoder.input,
	outputs=auto_encoder.get_layer("encoded").output)

# quantify the contents of our input images using the encoder
print("[INFO] encoding images...")
features_retrieved = encoder.predict(test_x)

[INFO] loading auto encoder model...
[INFO] encoding images...
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 115ms/step


## Perform search and retrieval based test images

In [25]:
query_indexes = list(range(0, test_x.shape[0]))
label_builder = list(np.unique(training_indexed["labels"]))
class_builder = {label_unique:[] for label_unique in label_builder}
recalls = copy.deepcopy(class_builder)
precisions = copy.deepcopy(class_builder)
# loop over the testing indexes
for i in query_indexes:
    queryFeatures = features_retrieved[i]
    results = perform_search(queryFeatures, training_indexed, max_results=5)
    labels_ret = [training_indexed["labels"][r[1]] for r in results]
    label_true = dataset[i].split("\\")[3]
    label_trues = [label_true for _ in labels_ret]
    recall = metric.recall_score(label_trues, labels_ret, average='weighted')
    precision = metric.precision_score(label_trues, labels_ret, average='weighted')
    recalls[label_true].append(recall)
    precisions[label_true].append(precision)

c:\Users\juamo\Desktop\PID_PROJECT\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\juamo\Desktop\PID_PROJECT\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\juamo\Desktop\PID_PROJECT\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\juamo\Desk

## Print the recall and precision values

In [26]:
print("recall values:")
comb_recall, comb_precision = [], []
for key in recalls.keys():
    average_val = np.average(recalls[key])
    print(key, average_val)
    comb_recall.append(average_val)
print("combined recall", np.average(comb_recall))

print("\nprecision values:")
for key in precisions.keys():
    average_val = np.average(precisions[key])
    print(key, average_val)
    comb_precision.append(average_val)
print("combined precision", np.average(comb_precision))

recall values:
benign 0.7838709677419355
malignant 0.8540145985401459
combined recall 0.8189427831410407

precision values:
benign 0.9838709677419355
malignant 1.0
combined precision 0.9919354838709677
